In [1]:
#this project was based on my own PC's jupyternotebook
# author: @Lyndon-N
import pandas as pd
import numpy as np
import jieba
import jieba.posseg as pseg
import wordcloud
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
# Target1
df = pd.read_excel('weibo.xlsx')
df=df[~(df['时间'].isnull())]    #删掉空行
df = df.sort_values(by='时间')
df

,时间,排名,热搜内容,地域,主题,上榜时间,最后时间,Unnamed: 7,Unnamed: 8,Unnamed: 9
897,2020-01-09 12:00:00,7.0,武汉不明原因肺炎病原体为新型冠状病毒,国内-湖北-武汉,疫情-知识,2020/1/9\n17:36,2020-01-09 08:38:00,NaN,NaN,NaN
896,2020-01-11 12:00:00,48.0,专家解读武汉不明原因肺炎,国内,专家|疫情-知识,2020-01-11 14:50:00,2020/1/11\n8:18,NaN,NaN,NaN
895,2020-01-14 12:00:00,8.0,泰国发现1例新型冠状病毒病例,国外-泰国,疫情-新增,2020-01-14 20:30:00,2020/1/14\n11:42,NaN,NaN,NaN
894,2020-01-15 12:00:00,14.0,武汉肺炎不排除有限人传人可能,国内-湖北-武汉,疫情-知识,2020-01-15 13:40:00,2020/1/15\n7:52,NaN,NaN,NaN
893,2020-01-15 12:00:00,8.0,泰国新型冠状病毒病例为武汉游客,国外-泰国,疫情-新增,2020-01-15 15:00:00,2020/1/15\n8:04,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
24,2020-02-21 12:00:00,44.0,西藏连续22天无新增病例,国内-西藏,疫情-稳定,2020-02-21 12:18:00,2020/2/21\n11:48,NaN,NaN,NaN
26,2020-02-21 12:00:00,48.0,威斯特丹号全员检测阴性,国外-游轮,疫情-新增|资讯-威斯特丹号,2020-02-21 12:04:00,2020/2/21\n11:12,NaN,NaN,NaN
27,2020-02-21 12:00:00,49.0,河南学校3月1日后开学,国内-河南,复工-开学,2020-02-21 12:38:00,2020/2/21\n7:20,NaN,NaN,NaN
14,2020-02-21 12:00:00,25.0,浙江新增28例新型肺炎,国内-浙江,疫情-新增,2020-02-21 13:52:00,2020/2/21\n9:38,NaN,NaN,NaN


In [3]:
theme = df['主题']
content = df['热搜内容']
location = df['地域']
time = df['时间']

In [4]:
entity_list = ['新型冠状病毒/8/nz', '冠状病毒/8/nz','疫情地图/8/nz','武汉肺炎/8/nz','人传人','病毒/8/nz','疫情/8/nz',
               '病例','新冠/8/nz', 'SARS/8/nz','隔离', '口罩', '卫健委/5/nt','双黄连/8/nz','埃博拉/3/nz','瑞德西韦/5/nz',
           '金银潭/5/nz','百步亭/5/nz','火神山/5/nz','雷神山/5/nz','张家界/5/ns', '小汤山/5/nz','威斯特丹号/5/nz','钻石公主号/5/nz',
               '方舱医院/5/nz','同济医院/3/nz','协和医院/3/nz','紫外线', '蝙蝠/3/nz','酒精','消毒','确诊','连闯/2/v',
               '康复/3/n','封城/3/n','汤圆/3/n', '武软/3/nt', '红十字会/3/nt', '马拉松/3/n', '小姐姐/3/r',
               '医护人员/5/nr','医疗队/5/nr','医院院长/5/nr','张定宇/5/nr','护士/5/nr','医生/5/nr',
               '钟南山/5/nr', '李兰娟/5/nr','李文亮/5/nr', '应勇/5/nr','王贺胜/5/nr', '王忠林/5/nr']
# for every entity, it is consist of [word content,word freq,word freq tag]
for w in entity_list:
    w_list = w.split('/')
    freq, tag = 1, None
    if len(w_list) > 1:
        freq = w_list[1]    # [word freq]
    if len(w_list) > 2:
        tag = w_list[2] # [word freq tag]
    #jieba.add_word(w)
    jieba.add_word(w_list[0], freq=freq, tag=tag)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Niuyi\AppData\Local\Temp\jieba.cache
Loading model cost 0.716 seconds.
Prefix dict has been built successfully.


In [5]:
# 总体字典
epidemic_v = OrderedDict()
epidemic_n = OrderedDict()
gov_v = OrderedDict()
gov_n = OrderedDict()

# 每日列表
daily_epidemic_v = []
daily_epidemic_n = []
daily_gov_v = []
daily_gov_n = []

now_time = time.iloc[0]
for i in range(len(theme)):
    theme_i = theme.iloc[i][:2]
    content_i = content.iloc[i]
    location_i = location.iloc[i]
    time_i = str(time.iloc[i]).split(" ")[0]
    if time_i != now_time:
        # 如果是同一天则加入同一个每日列表
        # 如果不是同一天则将该行数组拼接为字符串作为字典值
        epidemic_v[time_i] = ' '.join(daily_epidemic_v)
        epidemic_n[time_i] = ' '.join(daily_epidemic_n)
        gov_v[time_i] = ' '.join(daily_gov_v)
        gov_n[time_i] = ' '.join(daily_gov_n)
        # 然后清空每日列表
        daily_epidemic_v = []
        daily_epidemic_n = []
        daily_gov_v = []
        daily_gov_n = []

    words = pseg.cut(content_i)
    for word, lexical in words:
        if len(word) > 1:
            if '疫情' in theme_i:
                if lexical.startswith('n'):
                    daily_epidemic_n.append(word)
                if lexical.startswith('v'):
                    daily_epidemic_v.append(word)
            if '政府' in theme_i:
                if lexical.startswith('n'):
                    daily_gov_n.append(word)
                if lexical.startswith('v'):
                    daily_gov_v.append(word)
    now_time = time_i

In [6]:
# list(epidemic_v.values())
# print(epidemic_v)
# print(epidemic_n)
# print(gov_v)
# print(gov_n)

In [7]:
# 所有疫情主题词云
text_epidemic_v_all = ' '.join(list(epidemic_v.values()))
text_epidemic_n_all = ' '.join(list(epidemic_n.values()))
text_gov_v_all = ' '.join(list(gov_v.values()))
text_gov_n_all = ' '.join(list(gov_n.values()))

font = 'C:\Windows\Fonts\DENG.ttf'
mask = plt.imread('china.jpg')
wc = wordcloud.WordCloud(background_color='white', font_path=font, mask=mask, max_words=20, collocations=False)
wc.generate(text_epidemic_v_all)
wc.to_file('./overall/疫情-动词-全部.jpg')
wc.generate(text_epidemic_n_all)
wc.to_file('./overall/疫情-名词-全部.jpg')
wc.generate(text_gov_v_all)
wc.to_file('./overall/政府-动词-全部.jpg')
wc.generate(text_gov_n_all)
wc.to_file('./overall/政府-名词-全部.jpg')

In [8]:
period = 10
text_epidemic_v_period = ''
text_epidemic_n_period = ''
text_gov_v_period = ''
text_gov_n_period = ''
title_period = list(epidemic_v.keys())[0]
for i, time in enumerate(list(epidemic_v.keys())):
    if (i + 1) % period == 0:
        # once day-num reached period,then output 
        title_period = title_period + '~' + time
        if text_epidemic_v_period.strip() != '':
            wc.generate(text_epidemic_v_period)
            wc.to_file('./period/疫情-动词-' + title_period + '.jpg')
        if text_epidemic_n_period.strip() != '':
            wc.generate(text_epidemic_n_period)
            wc.to_file('./period/疫情-名词-' + title_period + '.jpg')
        if text_gov_v_period.strip() != '':
            wc.generate(text_gov_v_period)
            wc.to_file('./period/政府-动词-' + title_period + '.jpg')
        if text_gov_n_period.strip() != '':
            wc.generate(text_gov_n_period)
            wc.to_file('./period/政府-名词-' + title_period + '.jpg')
        text_epidemic_v_period = ''
        text_epidemic_n_period = ''
        text_gov_v_period = ''
        text_gov_n_period = ''
        title_period = time

    print(i, time)
    print('     疫情-动词', epidemic_v[time])
    print('     疫情-名词', epidemic_n[time])
    print('     政府-动词', gov_v[time])
    print('     政府-名词', gov_n[time])
    text_epidemic_v_period += ' ' + epidemic_v[time]
    text_epidemic_n_period += ' ' + epidemic_n[time]
    text_gov_v_period += ' ' + gov_v[time]
    text_gov_n_period += ' ' + gov_n[time]

0 2020-01-09
     疫情-动词 
     疫情-名词 
     政府-动词 
     政府-名词 
1 2020-01-11
     疫情-动词 不明
     疫情-名词 武汉 原因 肺炎 病原体 新型冠状病毒
     政府-动词 
     政府-名词 
2 2020-01-14
     疫情-动词 
     疫情-名词 
     政府-动词 
     政府-名词 
3 2020-01-15
     疫情-动词 发现
     疫情-名词 泰国 新型冠状病毒 病例
     政府-动词 
     政府-名词 
4 2020-01-16
     疫情-动词 排除 可能
     疫情-名词 武汉肺炎 传人 泰国 新型冠状病毒 病例 武汉 游客
     政府-动词 
     政府-名词 
5 2020-01-18
     疫情-动词 确认
     疫情-名词 日本 首例 新型冠状病毒 病例
     政府-动词 
     政府-名词 
6 2020-01-19
     疫情-动词 新增
     疫情-名词 武汉 新型冠状病毒 肺炎 病例
     政府-动词 
     政府-名词 
7 2020-01-20
     疫情-动词 新增
     疫情-名词 武汉 新型冠状病毒 肺炎 病例
     政府-动词 
     政府-名词 
8 2020-01-21
     疫情-动词 发现 发热 发布 高发 观察 新增 确诊 感染 传染 确诊 感染
     疫情-名词 浙江 武汉 症状 患者 武汉 卫健委 病毒性 肺炎 注意事项 深圳 新型冠状病毒 肺炎 病例 武汉 肺炎 广东 新型冠状病毒 肺炎 病例 新型冠状病毒 来源 北京 新型冠状病毒 肺炎 病例
     政府-动词 加强 可疑 筛查
     政府-名词 上海 病例
9 2020-01-22
     疫情-动词 严防 发热
     疫情-名词 春运 新型冠状病毒 新型冠状病毒 肺炎 新型冠状病毒 传染性 SARS 武汉 门诊 名单
     政府-动词 纳入
     政府-名词 新型冠状病毒 肺炎 法定 传染病
10 2020-01-23
     疫情-动词 确诊 死亡 存在 可能 暂无 出现 确诊 确诊
     疫情-名词 肺炎 新型冠状

In [9]:
df = pd.read_csv(open("DXYRumors.csv", encoding='utf8'), sep=',',dtype=str)
df

,_id,id,title,mainSummary,summary,body,sourceUrl,rumorType,crawlTime
0,5e61bd6be888d2a2299c3126,184,疫情期间大强度锻炼可提高抵抗力？,国家体育总局体育科学研究所研究员徐建方：长时间大强度的运动，会导致身体机能失调，免疫功能下降,NaN,长时间大强度的运动，会导致身体机能失调，免疫功能下降，并且运动损伤风险增加。因此，特别忌讳平...,NaN,0,2020-03-06 11:03:02.315
1,5e61bd6be888d2a2299c3127,182,吃降压药会增加感染病毒的风险？,丁香医生团队辟谣：目前没有任何动物和临床研究数据证实这种说法的可靠性,NaN,因现有研究显示ACE2是新型冠状病毒入侵人体的关键，网传服用ACEI（血管紧张素转化酶抑制剂...,NaN,0,2020-03-06 11:03:02.315
2,5e5e14bd91153bfddf437eee,184,疫情期间大强度锻炼可提高抵抗力？,国家体育总局体育科学研究所研究员徐建方：长时间大强度的运动，会导致身体机能失调，免疫功能下降,NaN,在居家防疫期间，为确保运动安全有效，运动强度必须适宜。强度过低，没有锻炼效果，但是长时间大强...,NaN,0,2020-03-03 16:26:32.037
3,5e5cbdc5c550814970f586ce,183,改用吸痰机后，武汉死亡人数减半？,刘良教授辟谣：没有干预过任何个例的临床治疗,NaN,有传闻称：「病患遗体解剖发现死者肺部出现大量痰栓，而痰栓是由呼吸机使用所产生，致人缺氧而死。...,NaN,0,2020-03-02 16:03:15.136
4,5e5cbd47c550814970f586a8,182,吃降压药会增加感染病毒的风险？,丁香医生团队辟谣：目前没有任何动物和临床研究数据证实这种说法的可靠性,NaN,因现有研究显示ACE2是新型冠状病毒入侵人体的关键，网传服用ACEI（血管紧张素转化酶抑制剂...,NaN,0,2020-03-02 16:01:08.847
...,...,...,...,...,...,...,...,...,...
279,5e3065acf8be3cb05ba2bf12,48,深圳要封城？,深圳市互联网信息办公室：是谣言,NaN,1月26日下午从深圳市新型冠状病毒感染的肺炎疫情防控工作领导小组办公室获悉，有关深圳市将要“...,NaN,0,2020-01-29 00:47:32.346
280,5e3065acf8be3cb05ba2bf13,49,日本派出千名医疗人员？,其实是误解,NaN,27日下午，网上流传一张日本媒体报道武汉肺炎疫情的截图，配文称“日本派了1000名医疗工作者...,NaN,0,2020-01-29 00:47:32.346
281,5e3065acf8be3cb05ba2bf14,50,带呼吸阀的口罩没有用？,丁香医生团队辟谣：口罩上有呼吸阀不用担心,NaN,呼吸阀的气流是单向向外的，不影响使用者的防护效果。但已经有症状的人，建议不要使用带有呼吸阀的...,NaN,0,2020-01-29 00:47:32.346
282,5e3065acf8be3cb05ba2bf15,51,熏醋可以预防新冠？,丁香医生团队辟谣：熏醋杀不死新型冠状病毒,NaN,一瓶醋里的醋酸含量最多也只有5%。将醋里的醋酸蒸发到空气中的做法，提升空气酸度的空间十分有限...,NaN,0,2020-01-29 00:47:32.346


In [10]:
title = df['title']

In [11]:
# 谣言分词字符串
rumor_v = ''
rumor_n = ''
rumor = ''

for i in range(len(title)):
    words = pseg.cut(title.iloc[i])
    for word, lexical in words:
        if len(word) > 1:
            rumor += ' ' + word
            if lexical.startswith('n'):
                rumor_n += ' ' + word
            if lexical.startswith('v'):
                rumor_v += ' ' + word

In [13]:
font = 'C:\Windows\Fonts\DENG.ttf'
mask = plt.imread('china.jpg')
wc = wordcloud.WordCloud(background_color='white', font_path=font, mask=mask, max_words=10, collocations=False)
wc.generate(rumor_v)
wc.to_file('./overall/谣言-动词.jpg')
wc.generate(rumor_n)
wc.to_file('./overall/谣言-名词.jpg')
wc.generate(rumor)
wc.to_file('./overall/谣言-全部.jpg')